<a href="https://colab.research.google.com/github/Kartik-Chaurasiya/Machine-Learning/blob/master/faq_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


In [3]:
import pandas as pd

In [17]:
#importing data
data = pd.read_excel('/gdrive/MyDrive/csv/faqs.xlsx')
data.head()

,ID,CATEGORY,PRODUCT/FEATURE,TOPIC,INFO,VIDEO,IMAGES,ADDITIONAL INFO
0,1.0,general,NaN,Will my policy divided in to 2 insurance company?,No,NaN,NaN,NaN
1,2.0,general,NaN,CWO is an Insurance Company?,No. CWO is not an insurance company and does n...,NaN,NaN,NaN
2,3.0,general,NaN,Who manages the Insurance Policy?,The Policy is serviced by the following three ...,NaN,NaN,NaN
3,4.0,general,NaN,Is CWO is making profits from the policies?,NO\nCWO is not a profit making Organization an...,NaN,NaN,NaN
4,5.0,enrolment,NaN,What is the procedure to enroll into this Heal...,A member is required to visit the designated w...,NaN,NaN,NaN


In [6]:
data.shape

(99, 8)

In [9]:
data.dtypes

ID                 float64
CATEGORY            object
PRODUCT/FEATURE    float64
TOPIC               object
INFO                object
VIDEO              float64
IMAGES             float64
ADDITIONAL INFO    float64
dtype: object

In [8]:
data.describe()

,ID,PRODUCT/FEATURE,VIDEO,IMAGES,ADDITIONAL INFO
count,10.00000,0.0,0.0,0.0,0.0
mean,5.50000,NaN,NaN,NaN,NaN
std,3.02765,NaN,NaN,NaN,NaN
min,1.00000,NaN,NaN,NaN,NaN
25%,3.25000,NaN,NaN,NaN,NaN
50%,5.50000,NaN,NaN,NaN,NaN
75%,7.75000,NaN,NaN,NaN,NaN
max,10.00000,NaN,NaN,NaN,NaN


In [10]:
data.isnull().sum()

ID                 89
CATEGORY            0
PRODUCT/FEATURE    99
TOPIC               0
INFO                0
VIDEO              99
IMAGES             99
ADDITIONAL INFO    99
dtype: int64

In [18]:
#droping empty columns
data = data.drop(['CATEGORY', 'ID', 'PRODUCT/FEATURE', 'VIDEO', 'IMAGES', 'ADDITIONAL INFO'], axis = 1)
data.head()

,TOPIC,INFO
0,Will my policy divided in to 2 insurance company?,No
1,CWO is an Insurance Company?,No. CWO is not an insurance company and does n...
2,Who manages the Insurance Policy?,The Policy is serviced by the following three ...
3,Is CWO is making profits from the policies?,NO\nCWO is not a profit making Organization an...
4,What is the procedure to enroll into this Heal...,A member is required to visit the designated w...


In [21]:
data.rename(columns={'TOPIC': 'Question',
                   'INFO': 'Answer'},
          inplace=True, errors='raise')

In [22]:
data.head()

,Question,Answer
0,Will my policy divided in to 2 insurance company?,No
1,CWO is an Insurance Company?,No. CWO is not an insurance company and does n...
2,Who manages the Insurance Policy?,The Policy is serviced by the following three ...
3,Is CWO is making profits from the policies?,NO\nCWO is not a profit making Organization an...
4,What is the procedure to enroll into this Heal...,A member is required to visit the designated w...


In [25]:
# this function is used to get printable results
def getResults(questions, fn):
    def getResult(q):
        answer, score, prediction = fn(q)
        return [q, prediction, answer, score]

    return pd.DataFrame(list(map(getResult, questions)), columns=["Q", "Prediction", "A", "Score"])

test_data = [
    "manager of Insurance Policy?",
    "Is CWO making profits?",
    "Is Health declaration compulsory?"
]

data

,Question,Answer
0,Will my policy divided in to 2 insurance company?,No
1,CWO is an Insurance Company?,No. CWO is not an insurance company and does n...
2,Who manages the Insurance Policy?,The Policy is serviced by the following three ...
3,Is CWO is making profits from the policies?,NO\nCWO is not a profit making Organization an...
4,What is the procedure to enroll into this Heal...,A member is required to visit the designated w...
...,...,...
94,How you ensure that there won’t be any hassle ...,We will provide 100% support in claim settleme...
95,How many procedure are covered in Day Care t...,"DAY CARE PROCEDURES: Covered under the policy,..."
96,Whether Health declaration is compulsory?,"Yes, it is necessary to declare all existing a..."
97,What are common reason of deduction / Repudia...,o Policy holder reasons: 1) Mistake in name / ...


string searching Levenshtein

In [28]:
!pip install Levenshtein

     |████████████████████████████████| 110 kB 8.3 MB/s 
     |████████████████████████████████| 2.0 MB 25.6 MB/s 


In [29]:
from Levenshtein import ratio

def getApproximateAnswer(q):
    max_score = 0
    answer = ""
    prediction = ""
    for idx, row in data.iterrows():
        score = ratio(row["Question"], q)
        if score >= 0.9: # I'm sure, stop here
            return row["Answer"], score, row["Question"]
        elif score > max_score: # I'm unsure, continue
            max_score = score
            answer = row["Answer"]
            prediction = row["Question"]

    if max_score > 0.3: # treshold is lowered
        return answer, max_score, prediction
    return "Sorry, I didn't get you.", max_score, prediction

getResults(test_data, getApproximateAnswer)

,Q,Prediction,A,Score
0,manager of Insurance Policy?,Who manages the Insurance Policy?,The Policy is serviced by the following three ...,0.819672
1,Is CWO making profits?,Is CWO is making profits from the policies?,NO\nCWO is not a profit making Organization an...,0.676923
2,Is Health declaration compulsory?,Whether Health declaration is compulsory?,"Yes, it is necessary to declare all existing a...",0.837838
